In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

## Capacity Analysis:

#### Capacity for 2-step schem based on proposed alphabets 

In [2]:
## Draw particular overall throughput + regions of dominance -- CHECK AXIS


SNR_MAC = np.linspace(-10,20,120)
SNR_BC = np.linspace(-10,20,120)
SNR_HSI = np.linspace(-10,20,120)
#SNR_MAC = np.linspace(-10,20,31)
#SNR_BC = np.linspace(-10,20,31)
#SNR_HSI = np.linspace(-10,20,31)
N = 4
j = 60
[cap, act_map] = pickle.load(open('Region_results.dat','r'))

#[cap, act_map] = pickle.load(open('Region_results_rand_phase.dat','r'))

tuples = np.asarray([(i,qq) for i in range(0, N+1) for qq in range(0, N-i+1) if i+2*qq<=N])[1:]

plt.figure()
plt.imshow(cap.transpose()[-1::-1,j,:],\
extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) # xmin, xmax, ymin, ymax
plt.colorbar()
plt.xlabel('SNR MAC [dB]')
plt.ylabel('SNR HSI [dB]')

plt.title('Achievable rates for XOR mapping and $\gamma_{BC}=$'+str(int(SNR_BC[j])))

levs = [0,1,2,3,4,5,6]
CS = plt.contour(act_map.transpose()[:,j,:],len(levs),\
                levels=levs, colors='k',\
                extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)])

for i in range(len(tuples)):
  if len(np.nonzero(act_map.transpose()[:,j,:] == i)[0] > 0):        
    x = np.nonzero(act_map.transpose()[:,j,:] == i)[1][5]
    y = np.nonzero(act_map.transpose()[:,j,:] == i)[0][5]
    Nb = tuples[i, 0]
    Ns = tuples[i, 1]
#    print Nb,Ns
    s = "".join(('(',str(Nb),',',str(Ns),')'))
    line = np.arange(np.min(SNR_MAC), np.max(SNR_MAC),(SNR_MAC[-1]-SNR_MAC[0])/len(SNR_MAC))
    #plt.text(line[x],line[y],s,backgroundcolor='white')
    plt.text(SNR_MAC[x],SNR_HSI[y],s,backgroundcolor='white')
  
    
    
#plt.savefig('Capacities_XOR_map_BC'+str(SNR_BC[j])+'.eps')
plt.savefig('Capacities_XOR_map_BC_rand_phase'+str(int(SNR_BC[j]))+'.eps')

plt.figure()
plt.imshow(act_map.transpose()[-1::-1,j,:],\
extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) # xmin, xmax, ymin, ymax

plt.colorbar()
plt.xlabel('SNR MAC [dB]')
plt.ylabel('SNR HSI [dB]')  
plt.title('Strategies mapping and $\gamma_{BC}=$'+str(int(SNR_BC[j])))
#plt.savefig('Strategies_mapping_XOR_map_BC_rand_phase'+str(SNR_BC[j])+'.eps')
#plt.savefig('Strategies_mapping_XOR_map_BC'+str(SNR_BC[j])+'.eps')
plt.show()



In [3]:
from scipy.optimize import linprog

[cap_BC, cap_HSI, cap_MAC] = pickle.load(open('Capacities.dat','r'))

l = 5
#i = 23
#j = 23
#k = 17
i = 85
j = 90
k = 65

r3 = cap_MAC[l,i,0]
r2As = cap_MAC[l,i,1]
r2Bs = cap_MAC[l,i,2]
r2b = cap_MAC[l,i,3]
r1As = cap_MAC[l,i,4]
r1Bs = cap_MAC[l,i,5]
r1b = cap_MAC[l,i,6]

Rbs = np.array([r2As,r2Bs]).min()
Rbss = np.array([r3, cap_BC[l,j]]).min()
Rs = np.array([r1As, r1Bs]).min()
Rb = np.array([r1b, cap_HSI[l,k]]).min()



c = [-1, -1]
A = [[2, 1], [1, 1], [1,0], [0,1]]

b = [Rbss, Rbs, Rs, Rb]
x0_bounds = (0, None)
x1_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds))
plt.figure()
plt.plot([0,r3/2.],[r3, 0],'k-', lw=2, label='MAC - Third Order')
#plt.plot([0,r2As],[r2As, 0],'r--', lw=2, label='Sencond Order')
plt.plot([0,r2Bs],[r2Bs, 0],'k:.', lw=2, label='MAC - Sencond Order')
plt.plot([Rs, Rs],[0, r1b],'k-.', lw=3, label='MAC - First Order - Superposed')
plt.plot([0,Rs],[r1b, r1b],'k--', lw=3, label='MAC - Fist Order - Basic')
plt.plot([0,cap_BC[l,j]/2.],[cap_BC[l,j], 0],'b-', lw=2, label='BC - Third Order')
plt.plot([0,Rs],[cap_HSI[l,k],cap_HSI[l,k]],'g--', lw=2, label='HSI - Fist Order - Basic')
plt.plot([res.items()[4][1][0]],[res.items()[4][1][1]],'co', ms=15)
#plt.plot([0.66],[1.5],'rx', ms=15,lw=5)
plt.legend()
plt.xlabel('$r_s$')
plt.ylabel('$r_b$') 
plt.title('Rate region and for $\gamma_{MAC=}$'+str(int(SNR_MAC[i]))+', $\gamma_{HSI=}$'+str(int(SNR_HSI[k]))+', $\gamma_{BC}$='+str(int(SNR_BC[j])))
plt.savefig('Rate_Regions_XOR_map_BC'+str(SNR_BC[k])+'.eps')
plt.show()


#### Comparison of capacites 3-step based on Gauss alphabets and 2-step besed on proposed alphabets

In [16]:
from capacity_eval import cap_eval_Gauss

#SNR_MAC = np.linspace(-10,20,120)
#SNR_BC = np.linspace(-10,20,120)
#SNR_HSI = np.linspace(-10,20,120)
SNR_MAC = np.linspace(-10,20,31)
SNR_BC = np.linspace(-10,20,31)
SNR_HSI = np.linspace(-10,20,31)


#[cap2_step, act_map] = pickle.load(open('Region_results.dat.dat','r'))
[cap2_step, act_map] = pickle.load(open('Region_results_rand_phase.dat','r'))

cap3 = 2./3*np.asarray([[[ np.max([cap_eval_Gauss(np.min([i,j,k])), np.min([cap_eval_Gauss(i),cap_eval_Gauss((j)/2.)])]) \
           for i in SNR_MAC] for j in SNR_BC] for k in SNR_HSI])
N = 4
j = 25

plt.figure()
plt.imshow(cap2_step.transpose()[-1::-1,j,:] - cap3[-1::-1,j,:],\
extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) # xmin, xmax, ymin, ymax
plt.colorbar()
plt.xlabel('SNR MAC [dB]')
plt.ylabel('SNR HSI [dB]')

plt.title('2-step - 3step XOR mapping and $\gamma_{BC}=$'+str(SNR_BC[j]))
plt.savefig('Capacities_XOR_map_2vs3_step_rand_phase_BC'+str(int(SNR_BC[j]))+'.eps')

plt.show()

In [3]:
from capacity_eval import cap_eval_Gauss

SNR_MAC = np.linspace(-10,20,120)
SNR_BC = np.linspace(-10,20,120)
SNR_HSI = np.linspace(-10,20,120)

#cap3A = 2./3*np.asarray([[[ cap_eval_Gauss(np.min([i,j,k])) \
#           for i in SNR_MAC] for j in SNR_BC] for k in SNR_HSI])
#cap3B = 2./3*np.asarray([[[ cap_eval_Gauss(np.min([cap_eval_Gauss(i),cap_eval_Gauss((j)/2.)])) \
#           for i in SNR_MAC] for j in SNR_BC] for k in SNR_HSI])

cap3 = 2./3*np.asarray([[[ np.max([cap_eval_Gauss(np.min([i,j,k])), np.min([cap_eval_Gauss(i),cap_eval_Gauss((j)/2.)])]) \
           for i in SNR_MAC] for j in SNR_BC] for k in SNR_HSI])
j = 25

plt.figure()
plt.imshow(cap3[-1::-1,j,:],\
extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) # xmin, xmax, ymin, ymax
plt.colorbar()
plt.xlabel('SNR MAC [dB]')
plt.ylabel('SNR HSI [dB]')
#plt.figure()
#plt.imshow(cap3A[-1::-1,j,:] > cap3B[-1::-1,j,:],\
#extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) # xmin, xmax, ymin, ymax
#plt.colorbar()
#plt.xlabel('SNR MAC [dB]')
#plt.ylabel('SNR HSI [dB]')


plt.title('3step XOR mapping and $\gamma_{BC}=$'+str(SNR_BC[j]))
plt.show()

In [9]:
from scipy.optimize import linprog
from capacity_eval import *

Nb = 2
Ns = 1
M = 1e4
#gMAC = 10
#gHSI = 4.7
#gHSI = 5.5
#gBC = 11.3

gMAC = 9.8
gHSI = 7.5
gBC = 12.8


cBC = eval_cap_BC(Nb, Ns, gBC, M)
cHSI = eval_cap_HSI(Nb, Ns, gHSI, M)
#cMAC = eval_MAC_rand_phase(Nb, Ns, gMAC, M1 = 300, M2 = 300)

cMAC = eval_MAC(Nb, Ns, gMAC, M)
#cMAC = np.array( [3.16984004,  2.5764985 ,  2.57675782,  1.51168887,  0.75597148,  0.75571739,  1.98341627])
#cBC = 3.4487448266329639
#cHSI = 1.613597068410328
r3 = cMAC[0]
r2As = cMAC[1]
r2Bs = cMAC[2]
r2b = cMAC[3]
r1As = cMAC[4]
r1Bs = cMAC[5]
r1b = cMAC[6]

Rbs = np.array([r2As,r2Bs]).min()
Rbss = np.array([r3, cBC]).min()
Rs = np.array([r1As, r1Bs]).min()
Rb = np.array([r1b, cHSI]).min()
Rs2 = r2b/2


#cap3 = 2./3* \
#np.max([cap_eval_Gauss(np.min([gMAC,gBC,gHSI])+1.76), np.min([cap_eval_Gauss(gMAC+1.76),cap_eval_Gauss((gBC+1.76)/2.)])]) 

#cap2_min_map = cap_eval_Gauss(np.min([gMAC, gBC, gHSI]))
cap2_JDF = np.min([cap_eval_Gauss(gMAC+3)/2,cap_eval_Gauss(gBC)/2])


cap3_min_map = 2./3* \
cap_eval_Gauss(np.min([gMAC,gBC,gHSI]))
          
cap3_JDF = 2./3* \
np.min([cap_eval_Gauss(gMAC),cap_eval_Gauss((gBC))/2.])


c = [-1, -1]
A = [[2, 1], [1, 1], [1,0], [0,1]]

b = [Rbss, Rbs, Rs, Rb]
x0_bounds = (0, None)
x1_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds))
plt.figure()
plt.plot([0,r3/2.],[r3, 0],'k--', lw=2, label='MAC - Third Order')
plt.plot([0,cBC/2.],[cBC, 0],'b--', lw=2, label='BC')
plt.plot([0,r2As],[r2As, 0],'k-.', lw=2, label='MAC - Second Order')
plt.plot([Rs, Rs],[0, r1b],'k-', lw=2, label='MAC - First Order')
plt.plot([0,Rs],[r1b, r1b],'k-', lw=2)
plt.plot([0,Rs],[cHSI,cHSI],'g-', lw=2, label='HSI')
plt.plot([res.items()[4][1][0]],[res.items()[4][1][1]],'co', ms=15, label='Maximum achievable $r_b+r_s$')
plt.plot([0.6],[1.4],'mx', ms=15,lw=5, label='Achived performance')
#plt.plot([0.71],[1.5],'b+', ms=15,lw=5, label='Achived performance - Turbo')
#plt.plot([0, cap3],[cap3,0],'--y', lw=3,label='Capacity 3-step - JDF')
plt.plot([0, cap3_min_map],[cap3_min_map,0],'-y', lw=2,label='Capacity 3-step - NC')
plt.plot([0, cap3_JDF],[cap3_JDF,0],'--m', lw=3,label='Capacity 3-step - Broadcast')
#plt.plot([0, cap2_min_map],[cap2_min_map,0],'-c', lw=2,label='Capacity 2-step - NC')
plt.plot([0, cap2_JDF],[cap2_JDF,0],'--r', lw=2,label='Capacity 2-step')

plt.legend(numpoints=1)
plt.xlabel('$r_s$')
plt.ylabel('$r_b$') 
plt.title('Rate region for ($N_b,N_s$)=('+str(Nb)+','+str(Ns)+'), $\gamma_{MAC=}$'+str(gMAC)+', $\gamma_{HSI=}$'+str(gHSI)+', $\gamma_{BC}$='+str(gBC))
plt.savefig('Rate_Regions_XOR_map_BC'+str(gBC)+'_MAC'+str(gMAC)+'_HSI'+str(gHSI)+'.eps')
plt.show()

#### Channel Parameterization Sensitivity - MAC

In [ ]:
from capacity_eval import *

Nb = 2
Ns = 1
gMAC = 15
M = 100
x = 20

X = np.linspace(-1,1,x)
Y = np.linspace(-1,1,x)

temp = Parallel(n_jobs=8)(delayed(eval_MAC_ef)(Nb,Ns,gMAC, M, i + 1j*j) for i in X for j in Y)
cap = np.asarray(temp)
s = 'MAC - Par. Sensitivity $\\gamma_{MAC}$:'+str(gMAC)+', '
title_list = ['3rd Order - $I(x;s_A^s, s_B^s,s_A^b\oplus s_B^b)$', \
              '2nd Order - $I(x;s_B^s, s_A^b\oplus s_B^b|s_A^s)$', \
              '2nd Order - $I(x;s_A^s, s_A^b\oplus s_B^b|s_B^s)$', \
              '2nd Order - $I(x;s_A^s, s_B| s_A^b\oplus s_B^b)$', \
              '1st Order - $I(x;s_A^s | s_B^s, s_A^b\oplus s_B^b)$', \
              '1st Order - $I(x;s_B^s | s_A^s, s_A^b\oplus s_B^b)$', \
              '1st Order - $I(x;s_A^b\oplus s_B^b|s_A^s,s_B^s)$']
              
              
for i in range(7):
  plt.figure()
  res = cap[:,i].reshape(x,x).transpose()
  plt.imshow(res,
  extent=[-1,1,-1,1]) # xmin, xmax, ymin, ymax
  plt.colorbar()
  region=plt.Circle((0.92,-0.19),0.05,color='k',fill=False,lw=3)
  fig = plt.gcf()
  fig.gca().add_artist(region)
  plt.title(s+title_list[i])
  plt.grid()
  plt.xlabel('$\Re[h_A/h_B]$')
  plt.ylabel('$\Im[h_A/h_B]$') 
  plt.savefig('MAC_Sensitivity_'+str(i)+'_'+str(gMAC)+'_Nb'+str(Nb)+'_Ns'+str(Ns)+'.eps')
plt.show()
    


In [ ]:
from capacity_eval import *

Nb = 4
Ns = 0
gMAC = 25
M = 100
x = 20

X = np.linspace(-1,1,x)
Y = np.linspace(-1,1,x)

temp = Parallel(n_jobs=8)(delayed(eval_MAC_ef)(Nb,Ns,gMAC, M, i + 1j*j) for i in X for j in Y)
cap = np.asarray(temp)

r3 = cap[:,0].reshape(x,x).transpose()
r2As = cap[:,1].reshape(x,x).transpose()
r2Bs = cap[:,2].reshape(x,x).transpose()
r2b = cap[:,3].reshape(x,x).transpose()
r1As = cap[:,4].reshape(x,x).transpose()
r1Bs = cap[:,5].reshape(x,x).transpose()
r1b = cap[:,6].reshape(x,x).transpose()
res = np.zeros([x,x],float)
for i in range(x):
    for j in range(x):    
      Rbs = np.array([r2As[i,j],r2Bs[i,j]]).min()
      Rbss = r3[i,j]
      Rs = np.array([r1As[i,j], r1Bs[i,j]]).min()
      Rb = r1b[i,j]
      c = [-1, -1]
      A = [[2, 1], [1, 1], [1,0], [0,1]]
      b = [Rbss, Rbs, Rs, Rb]
      x0_bounds = (0, None)
      x1_bounds = (0, None)
      r = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds))
      res[i,j] = r.items()[4][1][0] + r.items()[4][1][1]
plt.imshow(res,
extent=[-1,1,-1,1]) # xmin, xmax, ymin, ymax
plt.colorbar()
region=plt.Circle((0.92,-0.19),0.05,color='k',fill=False,lw=3)
fig = plt.gcf()
fig.gca().add_artist(region)
plt.title('Maximum achievable rate as function of h')
plt.grid()
plt.xlabel('$\Re[h_A/h_B]$')
plt.ylabel('$\Im[h_A/h_B]$') 
plt.show()


#### Sensitivity to Direct Link

In [69]:
from capacity_eval import *

Nb = 2
Ns = 1
#SNR_BC = 11.3
gHSI = 8
M = 10000
x = 20

X = np.linspace(-1,1,x)
Y = np.linspace(-1,1,x)

#temp = Parallel(n_jobs=8)(delayed(eval_cap_overall)(Nb,Ns,SNR_HSI, SNR_MAC, M, i + 1j*j) for i in X for j in Y)
temp = Parallel(n_jobs=8)(delayed(eval_HSI_gen)(Nb, Ns, gHSI, M, i + 1j*j) for i in X for j in Y)
cap = np.asarray(temp)
plt.figure()
res = cap.reshape(x,x).transpose()
plt.imshow(res,
extent=[-1,1,-1,1]) # xmin, xmax, ymin, ymax
plt.colorbar()
plt.title(title_list[i])
plt.grid()
plt.xlabel('$\Re[\\alpha]$')
plt.ylabel('$\Im[\\alpha]$') 
plt.title('Capacity as function of DL for HSI='+str(gHSI)+', Nb='+str(Nb)+', Ns='+str(Ns))
plt.savefig('Direct_link_sensitivity_HSI'+str(gHSI)+'_Nb'+str(Nb)+'_Ns'+str(Ns)+'.eps')
plt.show()

In [4]:
from scipy.optimize import linprog
from capacity_eval import *

Nb = 2
Ns = 1
M = 1e4
gMAC = 16.8
gHSI = 4.7
gBC = 11.3
alpha = 1

#end to end (slightly worse than given first orders)
#cE2E = eval_cap_overall(Nb, Ns, gHSI, gBC, M, hD = 0) 
cE2E = eval_overall_rand_phase(Nb, Ns, gHSI, gBC, M1 = 300, M2 = 300, alpha=0.8)

cMAC = eval_MAC_rand_phase(Nb, Ns, gMAC, M1 = 300, M2 = 300)
#cMAC = eval_MAC(Nb, Ns, gMAC, M)

cBC = eval_cap_BC(Nb, Ns, gBC, M)

cHSI = eval_cap_HSI(Nb, Ns, gHSI, M)
#cHSI = eval_HSI_gen(Nb, Ns, gHSI, M, 0)
#cHSI = eval_HSI_rand_phase(2, 1, gHSI, M1 = 300, M2 = 300, alpha = 0.8)

#cMAC = eval_MAC(Nb, Ns, gMAC, M)
#cMAC = np.array( [3.16984004,  2.5764985 ,  2.57675782,  1.51168887,  0.75597148,  0.75571739,  1.98341627])
#cBC = 3.4487448266329639
#cHSI = 1.613597068410328
r3 = cMAC[0]
r2As = cMAC[1]
r2Bs = cMAC[2]
r2b = cMAC[3]
r1As = cMAC[4]
r1Bs = cMAC[5]
r1b = cMAC[6]

Dr2 = cE2E[0]
Dr1b = cE2E[1]
Dr1s = cE2E[2]

#Rbs = np.array([r2As,r2Bs, Dr2]).min()
#Rbss = r3
#Rs = np.array([r1As, r1Bs, Dr1s]).min()
#Rb = np.array([r1b, Dr1b]).min()
Rbs = np.array([r2As,r2Bs]).min()
Rbss = np.array([r3,cBC]).min()
Rs = np.array([r1As, r1Bs]).min()
Rb = np.array([r1b, cHSI]).min()


#cap3 = 2./3* \
#np.max([cap_eval_Gauss(np.min([gMAC,gBC,gHSI])+1.76), np.min([cap_eval_Gauss(gMAC+1.76),cap_eval_Gauss((gBC+1.76)/2.)])]) 

cap3_min_map = 2./3* \
cap_eval_Gauss(np.min([gMAC,gBC,gHSI]))
          
cap3_JDF = 2./3* \
cap_eval_Gauss(np.min([cap_eval_Gauss(gMAC),cap_eval_Gauss((gBC))/2.]))


c = [-1, -1]
A = [[2, 1], [1, 1], [1,0], [0,1]]

b = [Rbss, Rbs, Rs, Rb]
x0_bounds = (0, None)
x1_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds))
plt.figure()
plt.plot([0,r3/2.],[r3, 0],'k--', lw=2, label='Relay - Third Order')
plt.plot([0,r2As],[r2As, 0],'k-.', lw=2, label='Relay - Second Order')
plt.plot([r1As, r1As],[0, r1b],'k-', lw=2, label='Relay - First Order')
plt.plot([0,r1As],[r1b, r1b],'k-', lw=2)
#plt.plot([0,Dr2],[Dr2, 0],'b--', lw=2, label='Dest - Second Order')
#plt.plot([Dr1s, Dr1s],[0, Dr1b],'b-', lw=2, label='Dest - First Order')
#plt.plot([0,Dr1s],[Dr1b, Dr1b],'b-', lw=2)

plt.plot([0,r1As],[cHSI,cHSI],'g-', lw=2, label='HSI - First Order')
plt.plot([0,cBC/2.],[cBC, 0],'g--', lw=2, label='BC - Third Order')

plt.plot([res.items()[4][1][0]],[res.items()[4][1][1]],'co', ms=15, label='Maximum achievable $r_b+r_s$')
plt.plot([0.5],[1],'mx', ms=15,lw=5, label='Achived performance')
#plt.plot([0, cap3],[cap3,0],'--y', lw=3,label='Capacity 3-step - JDF')
plt.plot([0, cap3_JDF],[cap3_JDF,0],'--y', lw=3,label='Capacity 3-step - JDF')
plt.plot([0, cap3_min_map],[cap3_min_map,0],'-m', lw=3,label='Capacity 3-step - Min Map')
plt.legend(numpoints=1)
plt.xlabel('$r_s$')
plt.ylabel('$r_b$') 
plt.title('Rate region and for $\gamma_{MAC=}$'+str(gMAC)+', $\gamma_{HSI=}$'+str(gHSI)+', $\gamma_{BC}$='+str(gBC))
plt.savefig('Overall_Rate_Regions_XOR_map_BC'+str(gBC)+'_MAC'+str(gMAC)+'_HSI'+str(gHSI)+'.eps')
plt.show()

In [2]:
run capacity_eval

## Throughput Analysis

In [7]:
SNR_HSI = np.arange(0,30,0.1)
SNR_MAC = np.arange(0,30,0.1)
len_frame = 768
N = 4  # Number of maximum bits per symbol in BC stage

tupples = np.asarray([(i,j) for i in range(0, N+1) for j in range(0, N-i+1) if i+2*j<=N])[1:] # all feassible Nb,Ns tupples
through = []
#through = [np.zeros([len(SNR_MAC), len(SNR_HSI)])] * (tupples.shape[0])
for [Nb, Ns] in tupples:
    through.append(Throughput_Eval_range(Nb, Ns, N, SNR_MAC, SNR_HSI, mapping='XOR', len_frame=len_frame))    

In [46]:
## Draw particular overall throughput + regions of dominance
run compare_QPSK

max_through = np.asarray(through).max(axis=0)
map_max = (through == max_through)
levs_total = np.arange(0,N+0.1,0.2)
region_advantage = np.zeros(np.shape(max_through))
#CS1 = plt.contour(max_through,len(levs),\
#      levels=levs_total, colors='y',\
#      extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)])
#zc = CS1.collections[0:len(levs_total):5] #
#plt.setp(zc, linewidth=4)
#plt.xlabel('SNR MAC [dB]')
#plt.ylabel('SNR HSI [dB]')
#plt.clabel(CS1, levs[0::2], inline=1, fmt='%1.1f', fontsize=14)

plt.imshow(max_through[-1::-1,:],\
extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) # xmin, xmax, ymin, ymax
plt.colorbar()
plt.xlabel('SNR MAC [dB]')
plt.ylabel('SNR HSI [dB]')


for i in range(0, len(through)):
    act_max = through[i]
    act_range = np.hstack([np.arange(0, i), np.arange(i+1, len(through))])
    other_max = np.max(np.asarray(np.asarray(through)[act_range,:,:]),axis=0)
    levs = [0]
    CS = plt.contour(act_max-other_max,len(levs),\
                levels=levs, colors='k',\
                extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)])
    zc = CS.collections[0] # Stronger isolines
    plt.setp(zc, linewidth=4)
    
    act_region = np.nonzero(max_through-other_max > 0)
    region_advantage[act_region] = max_through[act_region]-other_max[act_region]

    
    # Text description of regions
    Nb = tupples[i, 0]
    Ns = tupples[i, 1]
    s = "".join(('(',str(Nb),',',str(Ns),')'))
    line = np.arange(np.min(SNR_MAC), np.max(SNR_MAC),(SNR_MAC[-1]-SNR_MAC[0])/len(SNR_MAC))
    act_advantage = np.zeros(np.shape(max_through))
    act_region = np.nonzero(max_through-other_max > 0)
    act_advantage[act_region] = max_through[act_region]-other_max[act_region]

    x = np.nonzero(act_advantage == np.max(act_advantage))[1][0]
    y = np.nonzero(act_advantage == np.max(act_advantage))[0][0]
    plt.text(line[x],line[y],s)
    

#levs = np.arange(0.1,1,0.4)
#CS = plt.contour(region_advantage,len(levs),\
#                levels=levs, colors='m',\
#                extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) 
#zc = CS.collections # Stronger isolines
#plt.setp(zc, linewidth=0.5)  
#plt.clabel(CS, levs, inline=1, fmt='%1.1f', fontsize=10)
plt.title('Throughput and strategy regions for Mod Sum')
plt.savefig('../results/MS_basic.svg')
plt.show()

SyntaxError: invalid syntax (<ipython-input-46-5ddd3b6c63b0>, line 2)

In [13]:
## Draw particular overall throughput + regions of dominance

max_through = np.asarray(through).max(axis=0)
map_max = (through == max_through)
levs_total = np.arange(0,N+0.1,0.2)
region_advantage = np.zeros(np.shape(max_through))
#CS1 = plt.contour(max_through,len(levs),\
#      levels=levs_total, colors='y',\
#      extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)])
#zc = CS1.collections[0:len(levs_total):5] #
#plt.setp(zc, linewidth=4)
#plt.xlabel('SNR MAC [dB]')
#plt.ylabel('SNR HSI [dB]')
#plt.clabel(CS1, levs[0::2], inline=1, fmt='%1.1f', fontsize=14)


plt.xlabel('SNR MAC [dB]')
plt.ylabel('SNR HSI [dB]')


for i in range(0, len(through)):
    act_max = through[i]
    act_range = np.hstack([np.arange(0, i), np.arange(i+1, len(through))])
    other_max = np.max(np.asarray(np.asarray(through)[act_range,:,:]),axis=0)
    levs = [0]
    CS = plt.contour(act_max-other_max,len(levs),\
                levels=levs, colors='k',\
                extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)])
    zc = CS.collections[0] # Stronger isolines
    plt.setp(zc, linewidth=4)
    
    # Text description of regions
    Nb = tupples[i, 0]
    Ns = tupples[i, 1]
    s = "".join(('(',str(Nb),',',str(Ns),')'))
    line = np.arange(np.min(SNR_MAC), np.max(SNR_MAC),(SNR_MAC[-1]-SNR_MAC[0])/len(SNR_MAC))
    act_advantage = np.zeros(np.shape(max_through))
    act_region = np.nonzero(max_through-other_max > 0)
    act_advantage[act_region] = max_through[act_region]-other_max[act_region]

    x = np.nonzero(act_advantage == np.max(act_advantage))[1][0]
    y = np.nonzero(act_advantage == np.max(act_advantage))[0][0]
    if (x > 0) or (y>0):
        plt.text(line[x],line[y],s)    
   
plt.title('Strategy region map for Mod Sum mapping')
plt.savefig('../results/MS_map.pdf')
plt.show()

In [15]:
def Through_e(Nb,Ns):
  print 'processing ', Nb, Ns
  t = 0  
  phi_axis = np.arange(0,2*np.pi,np.pi/60.)
  for phi in phi_axis:
    h_act = np.exp(1j * phi)
#    print phi, h_act
    t += Throughput_Eval_range_all(Nb, Ns, SNR_MAC, SNR_HSI, SNR_BC, 'XOR',len_frame, h_act, D='single')
  return t/float(len(phi_axis))

SNR_HSI = np.arange(0,30,2)
SNR_MAC = np.arange(0,30,2)
#SNR_BC = np.arange(15,21,1)
SNR_BC = np.array([20])
len_frame = 768
N = 4 # Number of maximum bits per symbol in BC stage
    
tupples = np.asarray([(i,j) for i in range(0, N+1) for j in range(0, N-i+1) if i+2*j<=N])[1:] # all feassible Nb,Ns tupples
through = []
avg = 1 # Averaging
par = 1 # Parralel evaluation
#through = [np.zeros([len(SNR_MAC), len(SNR_HSI)])] * (tupples.shape[0])
if avg == 0:
  if par == 0:
    for [Nb, Ns] in tupples:
      through.append(Throughput_Eval_range_all(Nb, Ns, SNR_MAC, SNR_HSI, SNR_BC, 'XOR',len_frame, h=1., D='single'))
  else:
    through = Parallel(n_jobs=8)(delayed(Throughput_Eval_range_all)\
        (Nb, Ns, SNR_MAC, SNR_HSI, SNR_BC, 'XOR', len_frame, h=1., D='single') for (Nb, Ns) in tupples)

elif avg == 1: # Uniform averaging 
  if par == 0:
    for [Nb, Ns] in tupples:  
      through.append(Through_e(Nb,Ns))
  else:
    through = Parallel(n_jobs=8)(delayed(Through_e)(Nb, Ns) for (Nb, Ns) in tupples)
#  through.append(Throughput_Eval_range_all(Nb, Ns, SNR_MAC, SNR_HSI, SNR_BC, 'XOR',len_frame, h=1., D='single'))


strategy = np.zeros([len(SNR_MAC), len(SNR_HSI), len(SNR_BC)],int)
for i in range(len(through)):
  act_range = np.hstack([np.arange(0, i), np.arange(i+1, len(through))])  
#  print np.asarray(through)[act_range,:,:]
  z,y,x = np.nonzero(through[i]>(np.max(np.asarray(through)[act_range,:,:,:], axis=0)))
  for j in range(len(x)):
    strategy[x[j], y[j],z[j]] = i
#print strategy
#print tupples

for i in range(len(SNR_BC)):
  plt.figure(figsize=(15, 15))
#  plt.figure()    
  max_through = np.asarray(through)[:,i,:,:].max(axis=0)
  map_max = (through == max_through)
  levs_total = np.arange(0,N+0.1,0.2)
 

  
    
  plt.imshow(max_through[-1::-1,:],\
  extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) # xmin, xmax, ymin, ymax  
#  extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)],vmin=0, vmax=4) # xmin, xmax, ymin, ymax  
  plt.colorbar()
  plt.xlabel('SNR MAC [dB]')
  plt.ylabel('SNR HSI [dB]')
  plt.title('Throughput for $\gamma_{BC}$='+str(SNR_BC[i])+'dB')


    
  for j in range(0, len(through)):
    act_max = through[j][i,:,:]
    act_range = np.hstack([np.arange(0, j), np.arange(j+1, len(through))])
    other_max = np.max(np.asarray(np.asarray(through)[act_range,i,:,:]),axis=0)
    levs = [0]
    CS = plt.contour(act_max-other_max,len(levs),\
                levels=levs, colors='k',\
                extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)])
    zc = CS.collections[0] # Stronger isolines
    plt.setp(zc, linewidth=4)

    
    # Text description of regions
    Nb = tupples[j, 0]
    Ns = tupples[j, 1]
    s = "".join(('(',str(Nb),',',str(Ns),')'))
    line = np.arange(np.min(SNR_MAC), np.max(SNR_MAC),(SNR_MAC[-1]-SNR_MAC[0])/float(len(SNR_MAC)))
    act_advantage = np.zeros(np.shape(max_through))
    act_region = np.nonzero(max_through-other_max > 0)
    act_advantage[act_region] = max_through[act_region]-other_max[act_region]

    x = np.nonzero(act_advantage == np.max(act_advantage))[1][0]
    y = np.nonzero(act_advantage == np.max(act_advantage))[0][0]
    plt.text(line[x],line[y],s)
  
  plt.xticks(np.arange(30))
  plt.yticks(np.arange(30))
  plt.grid(None)
  plt.savefig('../XOR_map_Througput_BC'+str(SNR_BC[i])+'.eps')
plt.show()

processing  0 1
processing  0 2
processing  1 0
processing  2 0
processing  4 0
processing  2 1
processing  3 0
processing  1 1


In [11]:
t1 = through

#### Reference adaptivity of JDF+NC

In [5]:
#run compare_QPSK #import Throughput_Eval_range_all_ref_3_slot_JDF, Throughput_Eval_range_all_ref_3_slot_min_map

SNR_HSI = np.arange(0,30,0.1)
SNR_MAC = np.arange(0,30,0.1)
#SNR_BC = np.arange(15,21,1)
SNR_BC = np.array([20])
len_frame = 768
N = 6 # Number of maximum bits per symbol in BC stage
Nb = 0

tupples = np.asarray([(0,j) for j in range(0, N)])[1:] # all feassible Nb,Ns tupples
through = []
#through = [np.zeros([len(SNR_MAC), len(SNR_HSI)])] * (tupples.shape[0])
for [Nb, Ns] in tupples:
  through.append(Throughput_Eval_range_all_ref_3_slot_JDF(Ns, SNR_MAC, SNR_HSI, SNR_BC, 'XOR',len_frame, h=1., D='single'))
  through.append(Throughput_Eval_range_all_ref_3_slot_min_map(Ns, SNR_MAC, SNR_HSI, SNR_BC, 'XOR',len_frame, h=1., D='single'))

strategy = np.zeros([len(SNR_MAC), len(SNR_HSI), len(SNR_BC)],int)
for i in range(len(through)):
  act_range = np.hstack([np.arange(0, i), np.arange(i+1, len(through))])  
#  print np.asarray(through)[act_range,:,:]
  z,y,x = np.nonzero(through[i]>(np.max(np.asarray(through)[act_range,:,:,:], axis=0)))
  for j in range(len(x)):
    strategy[x[j], y[j],z[j]] = i
#print strategy
#print tupples

for i in range(len(SNR_BC)):
# plt.figure(figsize=(15, 15))
  plt.figure()    
  max_through_ref = np.asarray(through)[:,i,:,:].max(axis=0)
  map_max = (through == max_through_ref)
  levs_total = np.arange(0,N+0.1,0.2)
 

  
    
  plt.imshow(max_through_ref[-1::-1,:],\
  extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)],vmin=0, vmax=4) # xmin, xmax, ymin, ymax  
  plt.colorbar()
  plt.xlabel('SNR MAC [dB]')
  plt.ylabel('SNR HSI [dB]')
  plt.title('Throughput for $\gamma_{BC}$='+str(SNR_BC[i])+'dB')


    
  for j in range(0, len(through)):
    act_max = through[j][i,:,:]
    act_range = np.hstack([np.arange(0, j), np.arange(j+1, len(through))])
    other_max = np.max(np.asarray(np.asarray(through)[act_range,i,:,:]),axis=0)
    levs = [0]
    CS = plt.contour(act_max-other_max,len(levs),\
                levels=levs, colors='k',\
                extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)])
    zc = CS.collections[0] # Stronger isolines
    plt.setp(zc, linewidth=4)

    
    # Text description of regions    
    Ns = tupples[j/2, 1]
    s = "".join(('(',str(Nb),',',str(Ns),')'))
    line = np.arange(np.min(SNR_MAC), np.max(SNR_MAC),(SNR_MAC[-1]-SNR_MAC[0])/float(len(SNR_MAC)))
    act_advantage = np.zeros(np.shape(max_through_ref))
    act_region = np.nonzero(max_through_ref-other_max > 0)
    act_advantage[act_region] = max_through_ref[act_region]-other_max[act_region]

    x = np.nonzero(act_advantage == np.max(act_advantage))[1][0]
    y = np.nonzero(act_advantage == np.max(act_advantage))[0][0]
    plt.text(line[x],line[y],s)
  
#  plt.xticks(np.arange(30))
#  plt.yticks(np.arange(30))
  plt.grid(None)
#  plt.savefig('../XOR_map_Througput_BC'+str(SNR_BC[i])+'.eps')
plt.show()

#### Comparison of reference and proposed - Throughput

In [ ]:
SNR_HSI = np.arange(0,20,1)
SNR_MAC = np.arange(0,20,1)
#SNR_BC = np.arange(14,30,2)
SNR_BC = np.array([6])
len_frame = 768
N = 8 # Number of maximum bits per symbol in BC stage

tupplesR = np.asarray([(0,j) for j in range(0, N)])[1:] # all feassible Nb,Ns tupples
throughR = []
#through = [np.zeros([len(SNR_MAC), len(SNR_HSI)])] * (tupples.shape[0])
for [Nb, Ns] in tupplesR:
  throughR.append(Throughput_Eval_range_all_ref_3_slot_JDF(Ns, SNR_MAC, SNR_HSI, SNR_BC, 'XOR',len_frame, h=1., D='single'))
  throughR.append(Throughput_Eval_range_all_ref_3_slot_min_map(Ns, SNR_MAC, SNR_HSI, SNR_BC, 'XOR',len_frame, h=1., D='single'))

strategyR = np.zeros([len(SNR_MAC), len(SNR_HSI), len(SNR_BC)],int)
for i in range(len(throughR)):
  act_range = np.hstack([np.arange(0, i), np.arange(i+1, len(throughR))])  
#  print np.asarray(through)[act_range,:,:]
  z,y,x = np.nonzero(throughR[i]>(np.max(np.asarray(throughR)[act_range,:,:,:], axis=0)))
  for j in range(len(x)):
    strategyR[x[j], y[j],z[j]] = i
#print strategy
#print tupples


tupples = np.asarray([(i,j) for i in range(0, N+1) for j in range(0, N-i+1) if i+2*j<=N])[1:] # all feassible Nb,Ns tupples
through = []
#through = [np.zeros([len(SNR_MAC), len(SNR_HSI)])] * (tupples.shape[0])
for [Nb, Ns] in tupples:
  through.append(Throughput_Eval_range_all(Nb, Ns, SNR_MAC, SNR_HSI, SNR_BC, 'MS',len_frame, h=1., D='single'))


strategy = np.zeros([len(SNR_MAC), len(SNR_HSI), len(SNR_BC)],int)
for i in range(len(through)):
  act_range = np.hstack([np.arange(0, i), np.arange(i+1, len(through))])  
#  print np.asarray(through)[act_range,:,:]
  z,y,x = np.nonzero(through[i]>(np.max(np.asarray(through)[act_range,:,:,:], axis=0)))
  for j in range(len(x)):
    strategy[x[j], y[j],z[j]] = i

for i in range(len(SNR_BC)):
# plt.figure(figsize=(15, 15))
  plt.figure()    
  max_through_ref = np.asarray(throughR)[:,i,:,:].max(axis=0)
  max_through = np.asarray(through)[:,i,:,:].max(axis=0)  
  
  plt.imshow(max_through[-1::-1,:] - max_through_ref[-1::-1,:],\
  extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)]) # xmin, xmax, ymin, ymax             
#extent=[np.min(SNR_MAC), np.max(SNR_MAC), np.min(SNR_HSI),np.max(SNR_HSI)],vmin=0, vmax=2) # xmin, xmax, ymin, ymax  
  plt.colorbar()
  plt.grid(None)
  plt.xlabel('SNR MAC [dB]')
  plt.ylabel('SNR HSI [dB]')
  plt.title('Throughput gain for $\gamma_{BC}$='+str(SNR_BC[i])+'dB')
  plt.savefig('../MS_advantage_map_Througput_BC'+str(SNR_BC[i])+'.eps')
plt.show()

#### Demodulation Mapping:

In [2]:
run compare_QPSK

In [16]:
print np.unique(act_map)
print tupples

[1 4 5 7]
[[0 1]
 [0 2]
 [1 0]
 [1 1]
 [2 0]
 [2 1]
 [3 0]
 [4 0]]


In [40]:
np.min([1,2,3])

1

In [13]:
cap_MAC[:,j,0]

array([ 0.99998958,  3.71407891,  0.49999253,  2.74283276,  1.98868272,
        6.2510432 ,  3.99162127,  8.90493341])

In [2]:
120/8


4